In [ ]:
!pip install vertexai

In [1]:
from vertexai.preview import rag
from vertexai.preview.generative_models import GenerativeModel, Tool
import vertexai
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

# Initialize Vertex AI API
PROJECT_ID = "your-project-id"  # Replace with your project ID
vertexai.init(project=PROJECT_ID, location="us-central1")

# Create a RAG Corpus
display_name = "test_corpus"
paths = ["gs://my_bucket/my_files_dir"]  # Path to your files in Google Cloud Storage

# Configure embedding model
embedding_model_config = rag.EmbeddingModelConfig(
    publisher_model="publishers/google/models/text-embedding-004"
)

rag_corpus = rag.create_corpus(
    display_name=display_name,
    embedding_model_config=embedding_model_config,
)

# Import files into the RagCorpus
rag.import_files(
    rag_corpus.name,
    paths,
    chunk_size=512,
    chunk_overlap=100,
    max_embedding_requests_per_min=900,
)

# Create a retrieval query
response = rag.retrieval_query(
    rag_resources=[rag.RagResource(rag_corpus=rag_corpus.name)],
    text="What is RAG and why it is helpful?",
    similarity_top_k=10,
)

# Print the retrieval response
print("Retrieval Response:", response)

# Create a RAG retrieval tool
rag_retrieval_tool = Tool.from_retrieval(
    retrieval=rag.Retrieval(source=rag.VertexRagStore(
        rag_resources=[rag.RagResource(rag_corpus=rag_corpus.name)],
        similarity_top_k=3,
    ))
)

# Create a Gemini model instance for generation
rag_model = GenerativeModel(
    model_name="gemini-1.5-flash-001",
    tools=[rag_retrieval_tool]
)

# Generate a response based on retrieved context
generation_response = rag_model.generate_content("What is RAG and why it is helpful?")
print("Generated Response:", generation_response.text)

# Setting up LangChain for enhanced question answering
system_prompt = (
    "You are an expert. Provide clear, concise answers based on the provided context. "
    "If the information is not found in the context, state that the answer is unavailable."
)

prompt_template = ChatPromptTemplate.from_messages(
    ("system", system_prompt),
    ("human", "{input}")
)

# Create a question-answering chain with LangChain
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3)
chain = create_stuff_documents_chain(llm, prompt_template)
rag_chain = create_retrieval_chain(retriever=None, chain=chain)  # Replace `None` with your retriever

# Ask a question using the RAG chain
final_response = rag_chain.invoke({"input": "What are mutual funds?"})
print("Final Response:", final_response)

ModuleNotFoundError: No module named 'vertexai'